# Anomaly Detection - TreeMap Exploration

This notebook demonstrates how to visualize anomalies with Treemap charts for static code analysis data using jQAssistant and Neo4j. 

<br>  

### References
- [jqassistant](https://jqassistant.org)
- [Neo4j Python Driver](https://neo4j.com/docs/api/python-driver/current)
- [Plotly Treemap Chart](https://plotly.com/python/treemaps/)

In [ ]:
import os
import typing
from typing import List, Tuple

from IPython.display import display
import pandas as pd
import numpy as np

from plotly import graph_objects as plotly_graph_objects
import plotly.colors as plotly_colors

In [ ]:
#The following cell uses the build-in %html "magic" to override the CSS style for tables to a much smaller size.
#This is especially needed for PDF export of tables with multiple columns.

In [ ]:
%%html
<style>
/* CSS style for smaller dataframe tables. */
.dataframe th {
    font-size: 8px;
}
.dataframe td {
    font-size: 8px;
}
</style>

In [ ]:
# Pandas DataFrame Display Configuration
pd.set_option('display.max_colwidth', 500)

In [ ]:
from sys import version as python_version
print('Python version: {}'.format(python_version))

from numpy import __version__ as numpy_version
print('numpy version: {}'.format(numpy_version))

from pandas import __version__ as pandas_version
print('pandas version: {}'.format(pandas_version))

from neo4j import __version__ as neo4j_version
print('neo4j version: {}'.format(neo4j_version))

from plotly import version as plotly_version
print('plotly version: {}'.format(plotly_version))

In [ ]:
# Please set the environment variable "NEO4J_INITIAL_PASSWORD" in your shell 
# before starting jupyter notebook to provide the password for the user "neo4j". 
# It is not recommended to hardcode the password into jupyter notebook for security reasons.
from neo4j import GraphDatabase, Driver

def get_graph_database_driver() -> Driver:
    driver = GraphDatabase.driver(
        uri="bolt://localhost:7687",
        auth=("neo4j", os.environ.get("NEO4J_INITIAL_PASSWORD")) # type: ignore
    )
    driver.verify_connectivity()
    return driver

In [ ]:
def query_cypher_to_data_frame(query: typing.LiteralString, parameters: typing.Optional[typing.Dict[str, typing.Any]] = None):
    records, summary, keys = driver.execute_query(query, parameters_=parameters)
    return pd.DataFrame([record.values() for record in records], columns=keys)

In [ ]:
# Base settings for Plotly Treemap

plotly_main_layout_base_settings = {
    "margin": {"t": 50, "l": 15, "r": 15, "b": 15},
}
plotly_treemap_layout_base_settings = dict(
    **plotly_main_layout_base_settings
)
plotly_bar_layout_base_settings = dict(
    **plotly_main_layout_base_settings
)
plotly_treemap_figure_show_settings = {
    "renderer": None,
    "width": 1080,
    "height": 1080,
}

plotly_treemap_marker_base_style = {
    "cornerradius": 5,
}

#  Hot_r, ice_r, Viridis_r, speed_r, haline_r, thermal_r, Plasma_r, solar_r, Electric_r, Blackbody_r, deep_r, Turbo_r, amp, Reds, Blackbody_r, RdGy_r, RdBu_r
plotly_treemap_marker_base_color_scale = dict(
    **plotly_treemap_marker_base_style,
    colorscale='Hot_r',
)

In [ ]:
def create_treemap_settings(data_frame: pd.DataFrame, element_path_column: str = 'elementPath', element_name_column: str = "elementName") -> plotly_graph_objects.Treemap:
    """
    Creates a Plotly Treemap with the given settings and data frame.
    data_frame : pd.DataFrame : The input data frame
    return :plotly_graph_objects.Treemap : The prepared Plotly Treemap
    """
    return plotly_graph_objects.Treemap(
        labels=data_frame[element_name_column],
        parents=data_frame['directoryParentPath'],
        ids=data_frame[element_path_column],
        customdata=data_frame[['fileCount', 'absoluteAnomalyScore', 'normalizedAuthorityRank', 'normalizedBottleneckRank', 'normalizedBridgeRank', 'normalizedHubRank', 'normalizedOutlierRank', 'elementPath']],
        hovertemplate='<b>%{label}</b><br>Highlighted anomalies: %{customdata[0]}<br>Anomaly Score: %{customdata[1]:.4f}<br>Authority: %{customdata[2]}, Bottleneck: %{customdata[3]}, Bridge: %{customdata[4]}, Hub: %{customdata[5]}, Outlier: %{customdata[6]}<br>Path: %{customdata[7]}',
        maxdepth=-1,
        root_color="lightgrey",
        marker=dict(**plotly_treemap_marker_base_style),
    )

In [ ]:
def remove_last_path_file_extension(file_path_elements: list) -> list:
    """
    Removes the file extension of the last element of the file path so that only the file name remains.
    file_path_elements : list : The list of file path elements where the last one contains the file name with extension
    return : list : The list of the directories + the file name without extension as last element.
    """
    if not file_path_elements:
        return ['']
    if len(file_path_elements) == 1:
        return [os.path.splitext(file_path_elements[0])[0]]
    return file_path_elements[:-1] + [os.path.splitext(file_path_elements[-1])[0]]

def join_path_elements(file_path_elements: list) -> list:
    """
    Joins the file path elements (and removes the file extension).
    file_path_elements : list : The list of levels to convert
    return : list : The list of directories
    """
    prepared_path_elements = remove_last_path_file_extension(file_path_elements)
    return ['/'.join(prepared_path_elements[:i+1]) for i in range(len(prepared_path_elements))]

def add_element_path_column(input_dataframe: pd.DataFrame, file_path_column: str, element_path_column: str = 'elementPath'):
    """
    Adds a directory column to the input DataFrame based on the file path column.
    input_dataframe : pd.DataFrame : The input DataFrame
    file_path_column : str : The name of the file path column
    directory_column : str : The name of the directory column to be added
    return : pd.DataFrame : The DataFrame with added directory column
    """
    if element_path_column in input_dataframe.columns:
        return input_dataframe # Column already exists
    
    input_dataframe.insert(0, element_path_column, input_dataframe[file_path_column].str.split('/').apply(join_path_elements))
    input_dataframe = input_dataframe.explode(element_path_column)
    return input_dataframe

def add_element_name_column(input_dataframe: pd.DataFrame, element_path_column: str = 'elementPath', element_name_column: str = 'elementName'):
    """
    Adds a directory name column to the input DataFrame based on the directory column.
    input_dataframe : pd.DataFrame : The input DataFrame
    directory_column : str : The name of the directory column
    directory_name_column : str : The name of the directory name column to be added
    return : pd.DataFrame : The DataFrame with added directory name column
    """
    if element_name_column in input_dataframe.columns:
        return input_dataframe # Column already exists
    
    splitted_directories = input_dataframe[element_path_column].str.rsplit('/', n=1)
    input_dataframe.insert(1, element_name_column, splitted_directories.apply(lambda x: (x[-1])))
    return input_dataframe

def add_parent_directory_column(input_dataframe: pd.DataFrame, element_path_column: str = 'elementPath', directory_parent_column: str = 'directoryParentPath'):
    """
    Adds a directory parent column to the input DataFrame based on the directory column.
    input_dataframe : pd.DataFrame : The input DataFrame
    directory_column : str : The name of the directory column
    directory_parent_column : str : The name of the directory parent column to be added
    return : pd.DataFrame : The DataFrame with added directory parent column
    """
    if directory_parent_column in input_dataframe.columns:
        return input_dataframe # Column already exists
    
    # Remove last path element from directory_column to get the directory_parent_column
    splitted_directories = input_dataframe[element_path_column].str.rsplit('/', n=1)
    input_dataframe.insert(1, directory_parent_column, splitted_directories.apply(lambda x: (x[0])))
    
    # Clear parent (set to empty string) when it equal to the directory
    input_dataframe.loc[input_dataframe[directory_parent_column] == input_dataframe[element_path_column], directory_parent_column] = ''
    return input_dataframe

def count_unique_aggregated_values(values: pd.Series):
    """
    Return the number of unique values from an array of array of strings.
    Meant to be used as an aggregation function for dataframe grouping.
    values : Series : The pandas Series of values
    return : int : The number of files
    """
    return len(np.unique(np.concatenate(values.to_list())))

In [ ]:
Archetypes = typing.Literal["Authority", "Bottleneck", "Bridge", "Hub", "Outlier"]
archetype_names: List[Archetypes] = ["Authority", "Bottleneck", "Bridge", "Hub", "Outlier"]

def get_archetype_column_name(archetype: Archetypes) -> str:
    """
    Returns the column name for the given archetype.
    archetype : Archetypes : The archetype name
    return : str : The column name for the given archetype
    """
    return f"normalized{archetype}Rank"

def get_archetype_index(archetype: Archetypes) -> int:
    """
    Returns the index of the given archetype.
    archetype : Archetypes : The archetype name
    return : int : The index of the given archetype
    """
    return archetype_names.index(archetype)

archetype_columns = [get_archetype_column_name(name) for name in archetype_names]

In [ ]:
def query_data() -> pd.DataFrame:
    query: typing.LiteralString = """
        MATCH (anomalyScoreStats:File&!Directory&!Archive)
        WHERE anomalyScoreStats.anomalyScore < 0
        ORDER BY anomalyScoreStats.anomalyScore ASCENDING
        LIMIT 150 // n largest negative anomaly scores as threshold
         WITH collect(anomalyScoreStats.anomalyScore)[-1] AS anomalyScoreThreshold
        MATCH (anomalyRankStats:File&!Directory&!Archive)
         WITH anomalyScoreThreshold
             ,max(anomalyRankStats.anomalyAuthorityRank)  AS maxAnomalyAuthorityRank
             ,max(anomalyRankStats.anomalyBottleneckRank) AS maxAnomalyBottleneckRank
             ,max(anomalyRankStats.anomalyBridgeRank)     AS maxAnomalyBridgeRank
             ,max(anomalyRankStats.anomalyHubRank)        AS maxAnomalyHubRank
             ,max(anomalyRankStats.anomalyOutlierRank)    AS maxAnomalyOutlierRank
        MATCH (anomalous:File&!Directory&!Archive)
        WHERE (anomalous.anomalyScore < anomalyScoreThreshold
           OR  anomalous.anomalyHubRank        IS NOT NULL
           OR  anomalous.anomalyAuthorityRank  IS NOT NULL
           OR  anomalous.anomalyBottleneckRank IS NOT NULL
           OR  anomalous.anomalyOutlierRank    IS NOT NULL
           OR  anomalous.anomalyBridgeRank     IS NOT NULL)
        OPTIONAL MATCH (project:Artifact|Project)-[:CONTAINS]->(anomalous)
          WITH *
              ,coalesce(project.name + '/', '')                     AS projectName
              ,coalesce(anomalous.fileName, anomalous.relativePath) AS fileName
        RETURN replace(projectName + fileName, '//', '/')   AS filePath
              ,CASE WHEN anomalous.anomalyScore < 0 THEN abs(anomalous.anomalyScore) ELSE 0 END AS absoluteAnomalyScore
              ,coalesce(toFloat(anomalous.anomalyAuthorityRank) / maxAnomalyAuthorityRank, 0)   AS normalizedAuthorityRank
              ,coalesce(toFloat(anomalous.anomalyBottleneckRank) / maxAnomalyBottleneckRank, 0) AS normalizedBottleneckRank
              ,coalesce(toFloat(anomalous.anomalyBridgeRank) / maxAnomalyBridgeRank, 0)         AS normalizedBridgeRank
              ,coalesce(toFloat(anomalous.anomalyHubRank / maxAnomalyHubRank), 0)               AS normalizedHubRank
              ,coalesce(toFloat(anomalous.anomalyOutlierRank) / maxAnomalyOutlierRank, 0)       AS normalizedOutlierRank
        ORDER BY filePath ASCENDING
        """
    return query_cypher_to_data_frame(query)

In [ ]:
def prepare_data_for_treemap(data: pd.DataFrame, debug: bool = False) -> pd.DataFrame:
    if debug:
        display("1. query result ---------------------")
        display(data)

    # 3. Add multiple rows for each file path containing all its directories paths in the new column 'elementPath'
    data = add_element_path_column(data, 'filePath', 'elementPath')

    if debug:
        display("3. added elementPath --------------")
        display(data)

    # Group the files by their directory and count the number of files of each directory (across all levels).
    common_named_aggregation = dict(
        absoluteAnomalyScore=pd.NamedAgg(column="absoluteAnomalyScore", aggfunc="mean"),
        normalizedAuthorityRank=pd.NamedAgg(column="normalizedAuthorityRank", aggfunc="max"),
        normalizedBottleneckRank=pd.NamedAgg(column="normalizedBottleneckRank", aggfunc="max"),
        normalizedBridgeRank=pd.NamedAgg(column="normalizedBridgeRank", aggfunc="max"),
        normalizedHubRank=pd.NamedAgg(column="normalizedHubRank", aggfunc="max"),
        normalizedOutlierRank=pd.NamedAgg(column="normalizedOutlierRank", aggfunc="max"),
    )

    data = data.groupby(['elementPath']).aggregate(
        filePaths=pd.NamedAgg(column="filePath", aggfunc=np.unique),
        firstFile=pd.NamedAgg(column="filePath", aggfunc="first"),
        maxAnomalyScore=pd.NamedAgg(column="absoluteAnomalyScore", aggfunc="max"),
        **common_named_aggregation
    )

    # Sort the grouped and aggregated entries by the name of the directory ascending and the anomaly score descending.
    # The author with the most commits will then be listed first for each directory.
    data = data.sort_values(by=['elementPath', 'absoluteAnomalyScore'], ascending=[True, False])
    data = data.reset_index()

    if debug:
        display("4. grouped by elementPath --------------")
        display(data)

    # Group the entries again now only by their directory path to get the aggregated number of anomalies and ranks.
    data = data.groupby('elementPath').aggregate(
        fileCount=pd.NamedAgg(column="filePaths", aggfunc=count_unique_aggregated_values),
        firstFile=pd.NamedAgg(column="firstFile", aggfunc="first"),
        maxAnomalyScore=pd.NamedAgg(column="maxAnomalyScore", aggfunc="max"),
        **common_named_aggregation
    )
    data = data.reset_index()

    if debug:
        display("5. grouped by directory path --------------")
        display(data)

    # Add the name of the directory (last '/' separated element) and the parent directory path to the table.
    data = add_element_name_column(data, 'elementPath', 'elementName')
    data = add_parent_directory_column(data, 'elementPath', 'directoryParentPath')

    if debug:
        display("6. added directory and parent name --------------")
        display(data)

    # Group finally by all columns except for the directory name, parent and path (first 3 columns) and pick the longest (max) directory path in case there are multiple.
    all_column_names_except_for_the_directory_path = data.columns.to_list()[3:]
    data = data.groupby(all_column_names_except_for_the_directory_path).aggregate(
        elementName=pd.NamedAgg(column="elementName", aggfunc=lambda names: '/'.join(names)),
        directoryParentPath=pd.NamedAgg(column="directoryParentPath", aggfunc="first"),
        elementPath=pd.NamedAgg(column="elementPath", aggfunc="last"),
    )

    # Reorder the column positions so that the directory path is again the first column. 
    all_column_names_with_the_directory_path_first = ['elementPath', 'directoryParentPath', 'elementName'] + all_column_names_except_for_the_directory_path
    data = data.reset_index()[all_column_names_with_the_directory_path_first]

    if debug:
        display("7. final grouping --------------")
        display(data)
        display("Statistics --------------")
        data.describe()
    
    return data

## 1. Anomalies

In [ ]:
driver = get_graph_database_driver()
anomaly_file_paths = query_data()
anomaly_file_paths = prepare_data_for_treemap(anomaly_file_paths)
display(anomaly_file_paths)

### 1.1 Average anomaly score per file directory

In [ ]:
data_to_display = anomaly_file_paths.copy()

figure = plotly_graph_objects.Figure(plotly_graph_objects.Treemap(
    create_treemap_settings(data_to_display),
    marker=dict(
        **plotly_treemap_marker_base_color_scale,
        colors=data_to_display['absoluteAnomalyScore'], 
        colorbar={"title": "score"},
    ),
))
figure.update_layout(
    **plotly_treemap_layout_base_settings, # type: ignore
    title='Average anomaly score per directory',
)
figure.show(**plotly_treemap_figure_show_settings)

### 1.2 Overview of all anomaly archetypes per directory

In [ ]:
def mutual_exclusive_ranks(data: pd.DataFrame) -> pd.DataFrame:
    """
    Modifies the input data frame to ensure that only one archetype rank is non-zero per row.
    The archetype with the highest normalized rank is retained, and others are set to zero.
    data : pd.DataFrame : The input data frame
    return : pd.DataFrame : The modified data frame with mutual exclusive ranks
    """
    modified_data = data.copy()
    
    for dataframe_index, row in modified_data.iterrows():
        index = typing.cast(int, dataframe_index)
        max_rank_value = 0
        max_rank_column = None
        
        for column in archetype_columns:
            if row[column] > max_rank_value:
                max_rank_value = row[column]
                max_rank_column = column
        
        for column in archetype_columns:
            if column != max_rank_column:
                modified_data.at[index, column] = 0
    
    return modified_data

In [ ]:
def interpolate_color(low: Tuple[int, int, int], high: Tuple[int, int, int], normalized_value: float) -> str:
    """Linear interpolation between two RGB tuples, returns rgba string."""
    
    def linear_interpolation_of_color_component(color_component: int) -> int:
        return int(low[color_component] + (high[color_component] - low[color_component]) * normalized_value)
    
    red = linear_interpolation_of_color_component(0)
    green = linear_interpolation_of_color_component(1)
    blue = linear_interpolation_of_color_component(2)
    return f"rgb({red},{green},{blue})"


def get_rank_color(rank: float, low: Tuple[int, int, int], high: Tuple[int, int, int]) -> str:
    """Return transparent if rank == 0, else interpolate between low and high."""
    if rank <= 0:
        return "rgb(255,255,255)"
    return interpolate_color(low, high, rank)


def combine_rank_colors(
    dataframe: pd.DataFrame,
    rank_columns: List[str],
    color_pairs: List[Tuple[Tuple[int, int, int], Tuple[int, int, int]]],
) -> List[str]:
    """Combine multiple ranks, using the first nonzero value's color."""
    combined: List[str] = []
    for _, row in dataframe.iterrows():
        color = "rgb(255,255,255)"
        for rank_col, (low, high) in zip(rank_columns, color_pairs):
            rank = row[rank_col]
            if rank > 0:
                color = get_rank_color(rank, low, high)
                break
        combined.append(color)
    return combined


def get_rank_color_for_archetype(dataframe: pd.DataFrame, archetype: Archetypes) -> List[str]:
    """Get combined rank colors for a specific archetype."""
    archetype_column_name = get_archetype_column_name(archetype)
    coloring_pair = get_coloring_pairs()[archetype_names.index(archetype)]
    return combine_rank_colors(dataframe, [archetype_column_name], [coloring_pair])


def get_coloring_pairs() -> List[Tuple[Tuple[int, int, int], Tuple[int, int, int]]]:
    """Define the coloring scheme for each archetype."""
    assert len(archetype_names) == 5, "Expected exactly 5 archetypes."
    return [
        ((222, 235, 247), (33, 113, 181)), # Authority, Red shades
        ((254, 230, 206), (217, 72, 1)),   # Bottleneck, Green shades
        ((239, 237, 245), (106, 81, 163)), # Bridge, Blue shades
        ((254, 224, 210), (165,15,21)),    # Hub, Orange shades
        ((240, 240, 240), (82, 82, 82)),   # Outlier, Purple shades
    ]

In [ ]:
data_to_display = mutual_exclusive_ranks(anomaly_file_paths)
# Optionally only keep rows where at least one archetype rank is greater than zero
data_to_display = data_to_display[data_to_display[archetype_columns].sum(axis=1) > 0]

coloring_pairs = get_coloring_pairs()
combined_colors = combine_rank_colors(data_to_display, archetype_columns, coloring_pairs)

figure = plotly_graph_objects.Figure()

figure.add_trace(plotly_graph_objects.Treemap(
    create_treemap_settings(data_to_display),
    marker=dict(
        **plotly_treemap_marker_base_style,
        line=dict(width=1, color="black"),
        showscale=False,
        colors=combined_colors,
    ),
    name="Anomalies",
    opacity=0.8
))

# Add dummy scatter traces for legend
for name, (low, high) in zip(archetype_names, coloring_pairs):
    bright_color = interpolate_color(low, high, 0.4)  # light tone for legend filling
    dark_color = interpolate_color(low, high, 1.0)  # darkest tone for legend outline
    figure.add_trace(plotly_graph_objects.Scatter(
        x=[None],
        y=[None],
        mode="markers",
        marker=dict(size=12, color=bright_color, line=dict(width=2, color=dark_color)),
        name=name,
        legendgroup=name,
        showlegend=True,
    ))

figure.update_layout(
    **plotly_treemap_layout_base_settings, # type: ignore
    title='Overview of all anomaly archetypes per directory',
    legend=dict(
        orientation="h", # horizontal legend (use "v" for vertical)
        yanchor="bottom",
        y=-0.12,
        xanchor="center",
        x=0.5
    )
)
figure.update_xaxes(visible=False)
figure.update_yaxes(visible=False)

figure.show(**plotly_treemap_figure_show_settings)

### 1.3a Archetype - Authority per directory

In [ ]:
def plot_single_archetype_treemap(archetype: Archetypes, data: pd.DataFrame):
    """
    Plots a treemap for the given archetype using the provided data.
    archetype : Archetypes : The archetype to plot
    data : pd.DataFrame : The input data frame
    """
    data_to_display = data.copy()
    data_to_display = data_to_display[data_to_display[archetype_columns].sum(axis=1) > 0]

    archetype_column_name = get_archetype_column_name(archetype)
    combined_colors = get_rank_color_for_archetype(data_to_display, archetype)

    figure = plotly_graph_objects.Figure(plotly_graph_objects.Treemap(
        create_treemap_settings(data_to_display),
        marker=dict(
            **plotly_treemap_marker_base_style,
            colors=combined_colors,
            line=dict(width=1, color="black"),
            colorbar={"title": "rank"},
        ),
    ))
    figure.update_layout(
        **plotly_treemap_layout_base_settings, # type: ignore
        title=f'Archetype "{archetype}" per directory',
    )
    figure.show(**plotly_treemap_figure_show_settings)

In [ ]:
plot_single_archetype_treemap("Authority", data_to_display)

### 1.3b Archetype - Bottleneck per directory

In [ ]:
plot_single_archetype_treemap("Bottleneck", anomaly_file_paths)

### 1.3c Archetype - Bridge per directory

In [ ]:
plot_single_archetype_treemap("Bridge", anomaly_file_paths)

### 1.3d Archetype - Hub per directory

In [ ]:
plot_single_archetype_treemap("Hub", anomaly_file_paths)

### 1.3e Archetype - Outlier per directory

In [ ]:
plot_single_archetype_treemap("Outlier", anomaly_file_paths)